{/* cspell:ignore leftrightarrow */}

# Bit-ordering in the Qiskit SDK

If you have a set of $n$ bits (or qubits), you'll usually label each bit $0
\rightarrow n-1$. Different softwares and resources must choose how they order
these bits both in computer memory and when displayed on-screen.

## Qiskit conventions

Here's how the Qiskit SDK orders bits in different scenarios.

### Quantum circuits

The `QuantumCircuit` class stores its qubits in a list
(`QuantumCircuit.qubits`). The index of a qubit in this list defines the
qubit's label.

In [5]:
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.circuit import Qubit

qc = QuantumCircuit(2)
qc.qubits[0]  # qubit "0"

Qubit(QuantumRegister(2, "q"), 0)

<Qubit register=(2, "q"), index=0>

### Circuit diagrams

On a circuit diagram, qubit $0$ is the topmost qubit, and qubit $n-1$ the
bottommost qubit. You can change this with the `reverse_bits` argument of
`QuantumCircuit.draw` (see [Change ordering in
Qiskit](#change-ordering-in-qiskit)).

In [6]:
qc.x(1)
qc.draw()

          
q_0: ─────
     ┌───┐
q_1: ┤ X ├
     └───┘

### Integers

When interpreting bits as a number, bit $0$ is the least significant bit, and
bit $n-1$ the most significant. This is helpful when coding because each bit has
the value $2^\text{label}$ (label being the qubit's index in
`QuantumCircuit.qubits`). For example, the following circuit execution ends
with bit $0$ being `0`, and bit $1$ being `1`. This is interpreted as the
decimal integer `2` (measured with probability `1.0`).

In [7]:
from qiskit.primitives import StatevectorSampler as Sampler

qc.measure_all()

job = Sampler().run([qc])
result = job.result()
print(f" > Counts: {result[0].data.meas.get_counts()}")

 > Counts: {'10': 1024}


### Strings

When displaying or interpreting a list of bits (or qubits) as a string, bit
$n-1$ is the leftmost bit, and bit $0$ is the rightmost bit. This is because we
usually write numbers with the most significant digit on the left, and in
Qiskit, bit $n-1$ is interpreted as the most significant bit.

For example, the following cell defines a `Statevector` from a string of
single-qubit states. In this case, qubit $0$ is in state $|+\rangle$, and
qubit $1$ in state $|0\rangle$.

In [8]:
from qiskit.quantum_info import Statevector

sv = Statevector.from_label("0+")
sv.probabilities_dict()

{np.str_('00'): np.float64(0.4999999999999999),
 np.str_('01'): np.float64(0.4999999999999999)}

This occasionally causes confusion when interpreting a string of bits, as you
might expect the leftmost bit to be bit $0$, whereas it usually represents bit
$n-1$.

### Statevector matrices

When representing a statevector as a list of complex numbers (amplitudes),
Qiskit orders these amplitudes such that the amplitude at index $x$ represents
the computational basis state $|x\rangle$.

In [9]:
print(sv[1])  # amplitude of state |01>
print(sv[2])  # amplitude of state |10>

(0.7071067811865475+0j)
0j


### Gates

Each gate in Qiskit can interpret a list of qubits in its own way, but
controlled gates usually follow the convention `(control, target)`.

For example, the following cell adds a controlled-X gate where qubit $0$ is
the control and qubit $1$ is the target.

In [10]:
from qiskit import QuantumCircuit

qc = QuantumCircuit(2)
qc.cx(0, 1)
qc.draw()

          
q_0: ──■──
     ┌─┴─┐
q_1: ┤ X ├
     └───┘

Following all the previously mentioned conventions in Qiskit, this CX-gate
performs the transformation $|01\rangle \leftrightarrow |11\rangle$, so has the
following matrix.

$$
\begin{pmatrix}
 1 & 0 & 0 & 0 \\
 0 & 0 & 0 & 1 \\
 0 & 0 & 1 & 0 \\
 0 & 1 & 0 & 0 \\
\end{pmatrix}
$$

## Change ordering in Qiskit

To draw a circuit with qubits in reversed order (that is, qubit $0$ at the
bottom), use the `reverse_bits` argument. This only affects the generated
diagram and does not affect the circuit; the X-gate still acts on qubit $0$.

In [11]:
from qiskit import QuantumCircuit

qc = QuantumCircuit(2)
qc.x(0)
qc.draw(reverse_bits=True)

          
q_1: ─────
     ┌───┐
q_0: ┤ X ├
     └───┘

You can use the `reverse_bits` method to return a new circuit with the
qubits' labels reversed (this does not mutate the original circuit).

In [12]:
qc.reverse_bits().draw()

          
q_0: ─────
     ┌───┐
q_1: ┤ X ├
     └───┘

Note that in this new circuit, the X-gate acts on qubit $1$.

## Next steps

<Admonition type="tip" title="Recommendations">
  -  See an example of using circuits in the [Grover's Algorithm](https://learning.quantum.ibm.com/tutorial/grovers-algorithm) tutorial.
  -  Explore the [QuantumCircuit API](/docs/api/qiskit/qiskit.circuit.QuantumCircuit#quantumcircuit-class) reference.
</Admonition>